### Separating Actions from conservation-adjacent texts
'Irrelevant' now represents non-actions, 'Relevant' represents action evidence.

In [1]:
from tqdm.notebook import tqdm
import pandas as pd

In [2]:
DEV = True

if DEV:
    model_name = "avsolatorio/GIST-small-Embedding-v0"
else:
    model_name = "avsolatorio/GIST-Embedding-v0"


model_name = 'jinaai/jina-embeddings-v2-small-en'

In [3]:
def import_labelled_data(path="data/level-1.5/merged/data.json"):
    data = pd.read_json(path, encoding="latin-1")
    return data


data = import_labelled_data(path="../../../data/level-1.5/merged/data.json", )



# train test split
from sklearn.model_selection import train_test_split

# 0.45 train, 0.15 val, 0.4 test
train_data, test_data = train_test_split(data, test_size=0.4, random_state=42)
train_data, val_data = train_test_split(train_data, test_size=0.25, random_state=42)

display(data.info())
data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8763 entries, 0 to 8762
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   url           8763 non-null   object 
 1   text          8763 non-null   object 
 2   relevance     8763 non-null   object 
 3   multiclasses  8763 non-null   object 
 4   score-lv1     167 non-null    float64
dtypes: float64(1), object(4)
memory usage: 342.4+ KB


None

,url,text,relevance,multiclasses,score-lv1
0,https://www.conservationevidence.com/synopsis/...,1 \n \n \n2 \n \n \nSubtidal Benthic Invertebr...,relevant,[Marine Invertebrates],NaN
1,https://www.conservationevidence.com/synopsis/...,\n \n \n Control of freshwater \n invasi...,relevant,"[Fish, Rivers and Lakes, Invasive]",NaN
2,https://www.conservationevidence.com/synopsis/...,1 \n \nGrassland Conservation \n2 \n \nGrassla...,relevant,[Grassland],NaN
3,https://www.conservationevidence.com/synopsis/...,\n \n \nii \n \n \n \n \n \n \n \nPrimate Co...,relevant,[Mammals],NaN
4,https://www.conservationevidence.com/synopsis/...,CONSERVATION EVIDENCE SERIES SYNOPSES\nTerrest...,relevant,[Mammals],NaN


In [4]:
from chunking import chunk_dataset_and_explode


# roughly 4 characters per token
max_len = 512

train_data = chunk_dataset_and_explode(train_data, max_len=max_len, overlap=int(max_len * 0.2))
test_data = chunk_dataset_and_explode(test_data, max_len=max_len, overlap=int(max_len * 0.2))
val_data = chunk_dataset_and_explode(val_data, max_len=max_len, overlap=int(max_len * 0.2))

  0%|          | 0/3942 [00:00<?, ?it/s]

  0%|          | 0/3506 [00:00<?, ?it/s]

  0%|          | 0/1315 [00:00<?, ?it/s]

In [5]:
train_data['relevance'].value_counts()

relevance
relevant      52217
irrelevant    28784
Name: count, dtype: int64

In [6]:
def stratified_sample(dataset,label_column: str = 'relevance',num_samples_per_label: int = 100):
    return (
        dataset
        .sample(frac=1,random_state=42)
        .groupby(label_column)[dataset.columns]
        .apply(lambda x: x.sample(min(num_samples_per_label,len(x)),random_state=42),include_groups=True).reset_index(drop=True)
    )


def sorted_stratified_sample(dataset,label_column: str = 'relevance', sorting_column: str = 'score-lv1', num_samples_per_label: int = 100):
    # get top num_samples_per_label samples per label
    return (
        dataset
        .sort_values(sorting_column,ascending=False)
        .groupby(label_column)[dataset.columns]
        .apply(lambda x: x.head(min(num_samples_per_label,len(x))),include_groups=True).reset_index(drop=True)
    )

if DEV:
    train_data = sorted_stratified_sample(train_data,label_column='relevance', sorting_column='score-lv1', num_samples_per_label=500)
    val_data = val_data.sample(100,random_state=42)
    test_data = test_data.sample(500,random_state=42)
else:
    train_data = train_data.sample(frac=1,random_state=42)
    val_data = val_data.sample(500,random_state=42)
    test_data = test_data.sample(frac=1,random_state=42)


In [7]:
train_data

,chunk_id,url,text,relevance,multiclasses,score-lv1
0,8725,http://xerces.org/sites/default/files/19-053_B...,ication and reference only. The appearance of ...,irrelevant,[],1.0
1,8653,https://yearofthesalmon.org/wp-content/uploads...,Adapting Paciﬁc salmon management systems to a...,irrelevant,[],1.0
2,8653,https://yearofthesalmon.org/wp-content/uploads...,ciﬁc Bering Sea British Columbia FAR = 1 - (pr...,irrelevant,[],1.0
3,8653,https://yearofthesalmon.org/wp-content/uploads...,25 42-45 29 47 41 24 46 40 39 30-34 23 35-36 3...,irrelevant,[],1.0
4,8653,https://yearofthesalmon.org/wp-content/uploads...,m 1960-1988 1988-2012 -0.2 0 0.2 0.4 0.6 0.8 C...,irrelevant,[],1.0
...,...,...,...,...,...,...
995,3420,https://www.conservationevidence.com/individua...,"le vaccines: Lipogen Forte V1, Renogen, Bayova...",relevant,[Marine],NaN
996,3420,https://www.conservationevidence.com/individua...,k of bacterial kidney disease when compared wi...,relevant,[Marine],NaN
997,2625,https://www.conservationevidence.com/individua...,Create skylark plots for bird conservationA st...,relevant,"[Birds, Farmland]",NaN
998,2625,https://www.conservationevidence.com/individua...,w. Overwinter counts of yellowhammer Emberiza ...,relevant,"[Birds, Farmland]",NaN


In [8]:
from datasets import Dataset

train_dataset = Dataset.from_pandas(train_data, split="train")
test_dataset = Dataset.from_pandas(test_data, split="test")
val_dataset = Dataset.from_pandas(val_data, split="val")

train_dataset

Dataset({
    features: ['chunk_id', 'url', 'text', 'relevance', 'multiclasses', 'score-lv1'],
    num_rows: 1000
})

In [9]:

#! had to modify FastFitTrainer to at /fastfit/train.py, line 879, to add trust_remote_code=True to the loading of 'accuracy' metrics
#! don't know why it's not default, since accuracy is the default in fastfit



#! IMPORTANT: another change in FastFitTrainer, also at line 879; comment out and replace the fixed version above
#! since load_metric is deprecated in favour of evaluate.load()
#! added functionality for sending in multiple metrics to evaluate at once
#! added macro averages for non-accuracy metrics too
#! essentially, copy the below code to replace line 879

### Insert into line 879.

```python
        # metric = load_metric(self.data_args.metric_name, experiment_id=uuid.uuid4())
        from evaluate import combine, load
        if type(self.data_args.metric_name) == str: # single metric name
            metrics = [load(self.data_args.metric_name, experiment_id=uuid.uuid4())]
        elif type(self.data_args.metric_name) == list: # compute multiple metrics
            metrics = [load(metric,experiment_id=uuid.uuid4()) for metric in self.data_args.metric_name]

        # You can define your custom compute_metrics function. It takes an `EvalPrediction` object (a namedtuple with a
        # predictions and label_ids field) and has to return a dictionary string to float.
        def compute_metrics(p: EvalPrediction):
            predictions = (
                p.predictions[0] if isinstance(p.predictions, tuple) else p.predictions
            )
            predictions = (
                np.squeeze(predictions)
                if self.is_regression
                else np.argmax(predictions, axis=1)
            )
            references = p.label_ids

            results = {}

            for metric in metrics:
                if metric.name != 'accuracy':
                    results.update(metric.compute(predictions=predictions, references=references,average='macro'))
                else:
                    results.update(metric.compute(predictions=predictions, references=references))

            return results
    ```

In [10]:
import torch
print(torch.cuda.is_available())
print(torch.cuda.current_device())
print(torch.cuda.get_device_name(0))

print(torch.version.cuda)

torch.cuda.empty_cache()


True
0
NVIDIA GeForce RTX 3090
12.1


In [11]:
from fastfit import FastFitTrainer

# same args as the huggingface TrainingArguments
if DEV:
    output_dir = f'models/relevance/dev/{model_name}'
else:
    output_dir = f'models/relevance/{model_name}'

trainer = FastFitTrainer(
    model_name_or_path=model_name,
    train_dataset=train_dataset,
    validation_dataset=val_dataset,
    test_dataset=test_dataset,
    output_dir=output_dir,
    overwrite_output_dir=True,
    label_column_name='relevance',
    text_column_name="text",
    num_train_epochs=10,
    per_device_train_batch_size=64,
    per_device_eval_batch_size=64,
    max_text_length=max_len,
    num_repeats=1,
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_strategy='epoch',
    metric_name=['precision','recall','f1','accuracy'],
    load_best_model_at_end=True,
    metric_for_best_model='recall',
    fp16=True,
)

print(trainer.model.device)

07/31/2024 13:09:17 - WARNING - fastfit.train - Process rank: 0, device: cuda:0, n_gpu: 1distributed training: True, 16-bits training: True


[WARNING|modeling_utils.py:4366] 2024-07-31 13:09:19,049 >> Some weights of BertModel were not initialized from the model checkpoint at jinaai/jina-embeddings-v2-small-en and are newly initialized: ['embeddings.position_embeddings.weight', 'encoder.layer.0.intermediate.dense.bias', 'encoder.layer.0.intermediate.dense.weight', 'encoder.layer.0.output.LayerNorm.bias', 'encoder.layer.0.output.LayerNorm.weight', 'encoder.layer.0.output.dense.bias', 'encoder.layer.0.output.dense.weight', 'encoder.layer.1.intermediate.dense.bias', 'encoder.layer.1.intermediate.dense.weight', 'encoder.layer.1.output.LayerNorm.bias', 'encoder.layer.1.output.LayerNorm.weight', 'encoder.layer.1.output.dense.bias', 'encoder.layer.1.output.dense.weight', 'encoder.layer.2.intermediate.dense.bias', 'encoder.layer.2.intermediate.dense.weight', 'encoder.layer.2.output.LayerNorm.bias', 'encoder.layer.2.output.LayerNorm.weight', 'encoder.layer.2.output.dense.bias', 'encoder.layer.2.output.dense.weight', 'encoder.layer.3

Running tokenizer on dataset to infer max length for both query and document:   0%|          | 0/1000 [00:00<?…

Running tokenizer on dataset to infer max length for both query and document:   0%|          | 0/100 [00:00<?,…

Running tokenizer on dataset to infer max length for both query and document:   0%|          | 0/500 [00:00<?,…

Running tokenizer on dataset:   0%|          | 0/1000 [00:00<?, ? examples/s]

Running tokenizer on dataset:   0%|          | 0/100 [00:00<?, ? examples/s]

Running tokenizer on dataset:   0%|          | 0/500 [00:00<?, ? examples/s]

cuda:0


In [12]:

#! another fastfit library modification
#! in /fastfit/train.py, line 971, change ignore_keys_for_eval from type set to a list
#! since it gets concatenated to a list later on
#! note that since we've added lines above, this is now line 981
#! the line beginning ignore_keys_for_eval={"doc_input_ids","doc_attention_mask","labels"}


model = trainer.train()

[WARNING|logging.py:328] 2024-07-31 13:09:31,865 >> BertSdpaSelfAttention is used but `torch.nn.functional.scaled_dot_product_attention` does not support non-absolute `position_embedding_type` or `output_attentions=True` or `head_mask`. Falling back to the manual attention implementation, but specifying the manual implementation will be required from Transformers version v5.0.0 onwards. This warning can be removed using the argument `attn_implementation="eager"` when loading the model.
[WARNING|modeling_utils.py:1198] 2024-07-31 13:09:32,854 >> Could not estimate the number of tokens of the input, floating-point operations will not be computed


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,4.831200,4.766787,0.649830,0.656520,0.652778,0.720000
2,4.292600,4.780195,0.770374,0.823186,0.783476,0.810000
3,4.192700,4.762451,0.785651,0.825216,0.799505,0.830000
4,4.153400,4.868021,0.788194,0.836885,0.803036,0.830000
5,4.137300,5.062438,0.780378,0.846525,0.789800,0.810000
6,4.133000,4.985996,0.797802,0.843734,0.812997,0.840000
7,4.127100,5.027970,0.795231,0.820396,0.805731,0.840000
8,4.125200,5.063736,0.797802,0.843734,0.812997,0.840000
9,4.125800,5.070460,0.795231,0.820396,0.805731,0.840000
10,4.127700,5.083606,0.784007,0.813546,0.795649,0.830000


***** train metrics *****
  epoch                    =       10.0
  total_flos               =        0GF
  train_loss               =     4.2246
  train_runtime            = 0:00:47.02
  train_samples            =       1000
  train_samples_per_second =    212.675
  train_steps_per_second   =      3.403


In [13]:
results = trainer.evaluate()

***** eval metrics *****
  epoch                   =       10.0
  eval_accuracy           =       0.81
  eval_f1                 =     0.7898
  eval_loss               =     5.0624
  eval_precision          =     0.7804
  eval_recall             =     0.8465
  eval_runtime            = 0:00:00.26
  eval_samples            =        100
  eval_samples_per_second =    371.271
  eval_steps_per_second   =      7.425


In [14]:
results

{'eval_loss': 5.062438488006592,
 'eval_precision': 0.7803776683087028,
 'eval_recall': 0.8465246067985794,
 'eval_f1': 0.7897997566102445,
 'eval_accuracy': 0.81,
 'eval_runtime': 0.2693,
 'eval_samples_per_second': 371.271,
 'eval_steps_per_second': 7.425,
 'epoch': 10.0,
 'eval_samples': 100}

In [15]:
print(f'Accuracy: {results["eval_accuracy"]}')

Accuracy: 0.81


In [16]:
results = trainer.test()

***** test metrics *****
  epoch                   =       10.0
  eval_accuracy           =       0.84
  eval_f1                 =     0.8389
  eval_loss               =     5.1147
  eval_precision          =     0.8394
  eval_recall             =     0.8483
  eval_runtime            = 0:00:01.09
  eval_samples_per_second =    455.739
  eval_steps_per_second   =      7.292
  test_samples            =        500


In [17]:


from os import makedirs, path
if not path.exists(f'models/relevance/dev/{model_name}'):
    makedirs(f'models/relevance/dev/{model_name}')

model.save_pretrained(f'models/relevance/dev/{model_name}')

if not DEV:
    model.save_pretrained(f'models/relevance/{model_name}')